In [30]:
import pandas as pd
PATH = "rijs_2011_all_filtered.csv"
all_data = pd.read_csv(PATH, encoding="utf-8")
all_data

,target_gene,which_CS,carbon_source_uptake,growth_rate,acetate_secretion,pts,zwf,gnd,pgi,"edd, eda",...,"glta, can",icd,suc,"sdh, fum",mdh,mae,pck,pcc,"pta, acka","acea, aceb"
0,WT,gal,2.00,0.18,0.11,2.10,0.61,0.35,1.48,0.26,...,1.48,0.53,0.23,1.18,2.09,0.04,0.74,0.45,0.11,0.95
1,AlsR,gal,2.26,0.19,0.03,2.12,0.57,0.37,1.54,0.20,...,1.40,0.32,0.00,1.09,2.18,0.00,0.92,0.49,0.03,1.09
2,AppY,gal,2.64,0.18,0.15,1.94,0.53,0.41,1.40,0.13,...,1.36,0.27,0.00,1.10,2.16,0.04,0.90,0.39,0.15,1.10
3,ArcA,gal,2.08,0.18,0.04,2.18,0.54,0.22,1.63,0.32,...,1.94,0.91,0.64,1.67,2.67,0.03,0.82,0.38,0.04,1.03
4,ArcB,gal,2.49,0.19,0.04,2.49,0.63,0.62,1.85,0.02,...,2.18,1.20,0.90,1.88,2.67,0.20,0.74,0.58,0.04,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,RseA,glu,6.29,0.53,5.31,6.53,1.84,1.26,4.67,0.57,...,1.55,1.55,0.84,0.84,0.56,0.28,0.22,2.02,4.78,0.00
186,RseB,glu,6.22,0.54,4.51,6.69,2.08,1.28,4.59,0.80,...,1.63,1.63,0.84,0.84,0.56,0.29,0.22,2.17,4.38,0.00
187,Pgi,glu,4.01,0.32,1.51,3.77,3.61,3.09,0.15,0.52,...,0.76,0.76,0.26,0.26,0.26,0.00,0.48,1.56,1.23,0.00
188,Zwf,glu,6.35,0.44,4.62,6.28,0.75,0.60,5.51,0.15,...,1.82,1.82,1.19,1.19,1.12,0.07,0.86,2.27,5.04,0.00


In [28]:
import json
reg2reacs = json.load(open("reg_id_to_reac_ids.json", 'r'))
cs2reac = json.load(open("carbon_source_to_reac_id.json", 'r'))

In [49]:
all_data = all_data[all_data["target_gene"].isin(list(reg2reacs.keys()))].reset_index(drop=True)

In [37]:
minmed = pd.read_csv("minmed_iML1515.csv").columns.to_list()

In [38]:
CS_ids = list(cs2reac.values())

In [39]:
genes = list(reg2reacs.keys())

In [54]:
forbidden_fluxes = ["pts", "zwf", "tktA", "tktB", "suc"]

flux2reac_gluc = {
    "growth_rate":["BIOMASS_Ec_iML1515_core_75p37M"],
    "carbon_source_uptake":["EX_glc__D_e_i_MEASURE"],
    "acetate_secretion":["EX_ac_e_o"],
    "gnd":["GND"],
    "pgi":["PGI_for"],
    "edd, eda":["EDD", "EDA"],
    "pfk, fba":["PFK", "FBA_for"], # different product in the paper (2 G3P instead of 1), but seem ok
    "tal":["TALA_for"],
    "gap, pgk":["GAPD_for", "PGK_rev", "PGM_rev"], # in paper: G3P>2PG; here PGM:2PG>3PG and PGK:3PG>13DPG and GAPD:G3P>13DPG
    "eno":["ENO_for"],
    "pyk":["PYK"], # ok but there's also PYK2,3,4,6 with different cofactors...
    "pdh":["PDH"],
    "glta, can":["CS", "ACONTa_for", "ACONTb_for"],
    "icd":["ICDHyr_for"],
    "sdh, fum":["SUCDi", "FUM_for"],
    "mdh":["MDH_for"],
    "mae":["ME1"],
    "pck":["PPCK"],
    "pcc":["PPC"],
    "pta, acka":["PTAr_for", "ACKr_rev"],
    "acea, aceb":["ICL", "MALS"]
}

flux2reac_galac = {
    "growth_rate":["BIOMASS_Ec_iML1515_core_75p37M"],
    "carbon_source_uptake":["EX_gal_e_i_MEASURE"],
    "acetate_secretion":["EX_ac_e_o"],
    "gnd":["GND"],
    "pgi":["PGI_for"],
    "edd, eda":["EDD", "EDA"],
    "pfk, fba":["PFK", "FBA_for"], # different product in the paper (2 G3P instead of 1), but seem ok
    "tal":["TALA_for"],
    "gap, pgk":["GAPD_for", "PGK_rev", "PGM_rev"], # in paper: G3P>2PG; here PGM:2PG>3PG and PGK:3PG>13DPG and GAPD:G3P>13DPG
    "eno":["ENO_for"],
    "pyk":["PYK"], # ok but there's also PYK2,3,4,6 with different cofactors...
    "pdh":["PDH"],
    "glta, can":["CS", "ACONTa_for", "ACONTb_for"],
    "icd":["ICDHyr_for"],
    "sdh, fum":["SUCDi", "FUM_for"],
    "mdh":["MDH_for"],
    "mae":["ME1"],
    "pck":["PPCK"],
    "pcc":["PPC"],
    "pta, acka":["PTAr_for", "ACKr_rev"],
    "acea, aceb":["ICL", "MALS"]
}

# TODO: check if perf AMN is good, if not remove the ones that might be problematic: pfk,fba + gap,pgk + pyk

In [67]:
import numpy as np
measured_fluxes = []
for key in flux2reac_gluc.keys():
    for flux in flux2reac_gluc[key]:
        measured_fluxes.append(flux)
    for flux in flux2reac_galac[key]:
        measured_fluxes.append(flux)

measured_fluxes = list(np.unique(measured_fluxes))

In [42]:
import cobra
model = cobra.io.read_sbml_model("iML1515_duplicated.xml")
iML1515_reactions = [reac.id for reac in model.reactions]

In [78]:
columns_amn_dataset_X_medium = minmed + CS_ids
columns_amn_dataset_X_genes = genes
columns_amn_dataset_X = columns_amn_dataset_X_medium + columns_amn_dataset_X_genes
columns_amn_dataset_Y = measured_fluxes
columns_amn_dataset_mask = iML1515_reactions

In [65]:
minmed_df = np.ones((len(all_data), len(minmed)))*50

varmed_df = np.zeros((len(all_data), len(CS_ids)))

genes_df = np.ones((len(all_data), len(genes)))

mask_df = np.zeros((len(all_data), len(iML1515_reactions)))

fluxes_df = np.zeros((len(all_data), len(measured_fluxes)))

In [69]:
for i in all_data.index:

    # fill varmed
    which_cs = all_data.iloc[i]["which_CS"]
    varmed_df[i][CS_ids.index(cs2reac[which_cs])] = 10

    # fill X_genes
    target_gene = all_data.iloc[i]["target_gene"]
    genes_df[i][genes.index(target_gene)] = 0

    # fill mask
    unreg_reactions = reg2reacs[target_gene]
    for unreg_reac in unreg_reactions:
        mask_df[i][iML1515_reactions.index(unreg_reac)] = 1

    for col in all_data.columns:
        if col == "which_CS":
            continue
        elif col == "target_gene":
            continue
        elif col in forbidden_fluxes:
            continue
        else:
            measured_value = all_data.iloc[i][col]

            if which_cs =="gal":
                reacs = flux2reac_galac[col]
            if which_cs == "glu":
                reacs = flux2reac_gluc[col]

            for reac in reacs:
                fluxes_df[i][measured_fluxes.index(reac)] = measured_value

In [83]:
columns_full = columns_amn_dataset_X + columns_amn_dataset_Y
full_df = pd.DataFrame(np.hstack([minmed_df, varmed_df, genes_df, fluxes_df]), columns=columns_full)
full_df.to_csv("rijs_iML1515_EXP.csv", index=False)

In [82]:
mask_df = pd.DataFrame(mask_df, columns=columns_amn_dataset_mask)
mask_df.to_csv("rijs_iML1515_EXP_mask.csv", index=False)

In [84]:
minmed_df.shape

(128, 23)

In [85]:
varmed_df.shape

(128, 2)

In [86]:
genes_df.shape

(128, 64)

In [87]:
fluxes_df.shape

(128, 31)